# L5 Process Inputs: Chaining Prompts


## Configuration
#### Chargement de la clé OpenAI et des librairies Python

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)


In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

![Chaining of prompt 01](../slides/chaining_prompts_01.jpg)

![Chaining of prompt 02](../slides/chaining_prompts_02.jpg)

![Chaining of prompt 03](../slides/chaining_prompts_03.jpg)

![Chaining of prompt 04](../slides/chaining_prompts_04.jpg)

![Chaining of prompt 05](../slides/chaining_prompts_05.jpg)

![Chaining of prompt 06](../slides/chaining_prompts_06.jpg)

![Chaining of prompt 07](../slides/chaining_prompts_07.jpg)

## Mettre en œuvre une tâche complexe avec plusieurs prompts

### Extraire les noms de produits et de catégories pertinents

In [ ]:
delimiter = "####"
system_message = f"""
Vous recevrez des demandes de service client.
La demande de service client sera délimitée par les caractères {delimiter}.
Affichez une liste d'objets Python, où chaque objet a le format suivant :
    'catégorie': <l'une des catégories suivantes : Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
OU
    'produits': <une liste de produits qui doivent se trouver dans les produits autorisés ci-dessous.>

Où les catégories et les produits doivent être trouvés dans la demande de service client.
Si un produit est mentionné, il doit être associé à la bonne catégorie dans la liste de 
produits autorisés ci-dessous. Si aucun produit ou catégorie n'est trouvé, 
affichez une liste vide.

Produits autorisés :

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Affichez uniquement la liste d'objets, sans rien d'autre.
"""
user_message_1 = f"""
Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap,
le reflex numérique. Parlez-moi aussi de vos téléviseurs."""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

In [ ]:
user_message_2 = f"""
Mon routeur ne fonctionne pas."""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

### Récupérer des informations détaillées sur les produits pour les produits et catégories extraits

In [ ]:
import json

with open('../products.json', 'r') as file:
    products = json.load(file)

In [ ]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

In [ ]:
print(get_product_by_name("TechPro Ultrabook"))

In [ ]:
print(get_products_by_category("Computers and Laptops"))

In [ ]:
print(user_message_1)

In [ ]:
print(category_and_product_response_1)

### Lire une chaîne Python dans une liste de dictionnaires Python

In [ ]:
import json 

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None   
    

In [ ]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

#### Récupérer des informations détaillées sur les produits et catégories pertinents

In [ ]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "produits" in data:
                products_list = data["produits"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "catégorie" in data:
                category_name = data["catégorie"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

In [ ]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

### Générer une réponse à la demande de l'utilisateur basée sur les informations détaillées sur les produits

In [ ]:
system_message = f"""
Vous êtes un assistant du service client pour un grand magasin d'électronique.
Répondez avec un ton amical et serviable, en donnant des réponses très concises.
Assurez-vous de poser à l'utilisateur des questions de suivi pertinentes. 
"""
user_message_1 = f"""
Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap, le reflex numérique.
Parlez-moi également de vos téléviseurs."""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""Relevant product information:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)